In [60]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import string
%matplotlib inline

# Config

In [61]:
MAX_LEN = 500 #due to the max length of the token [most BERT agree 512 token] => 512 without features or labels in it
# # batch size = 16, 32
batch_size = 4
premodel_list = ['xlm-roberta-base','xlm-roberta-large']
XLMR_chosen_premodel = premodel_list[1]
# # Model_path
XLMR_path = "./XLMR_Model.h5"

# Load GPU

In [62]:
if torch.cuda.is_available(): 
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3090


# Load dataset or news

In [67]:
# Example for csv with only one col

# df_crime_test = pd.read_csv('./example.csv') # Edit path based on the directory of dataset
# test_array = df_crime_test.values
# X_test = np.array(test_array)

In [65]:
# the value in iloc is depending on news column for dataset with multi-columns

# df_crime_test = pd.read_csv('./1_Test.csv') # Edit path based on the directory of dataset
# X_test = df_crime_test.iloc[:,3:4].values

In [64]:
# df_crime_test

,News
0,อัจฉริยะ งัดหลักฐานฟันธงคดีแตงโม ฆาตกรรม ไม่ได...


In [66]:
# Example of classifying only one news

test = "อัจฉริยะ งัดหลักฐานฟันธงคดีแตงโม ฆาตกรรม ไม่ได้ตกจากท้ายเรือ เมื่อเวลา 14.00 น. ที่ รร.อัณณา นาวา ปากเกร็ด นายอัจฉริยะ เรืองรัตนพงษ์ จัดแถลงข่าว กรณีเปิดหลักฐาน แตงโม ไม่ได้ตกท้ายเรือ และข้อพิสูจน์ว่าตำรวจ มีความผิด ฐานปฏิบัติหน้าที่ไม่ชอบ ม.157 ว่า ขณะนี้ตำรวจยังไม่สามารถตอบเรื่องบาดแผลข้างขวา ซึ่งเคยออกมาระบุว่า ไม่ใช่สาเหตุการเสียชีวิตได้ โดย พ.อ.นพ. ธวัชชัย กาญจนรินทร์” อดีตศัลยแพทย์โรงพยาบาลพระมงกุฎเกล้า ได้วิเคราะห์ สรุปได้ว่า เกิดจากของที่คมมาก หรือ ถ้าเกิดในแม่น้ำจะพุ่งกระจายหยุดยาก ชมรมฯ เห็นด้วยกับบทความนี้ ว่า แผลขวาด้านใน ไม่สามารถเกิดจากใบพัดหรือฟินเรือได้ทั้งนี้ นายอัจฉริยะ ได้เปิดคลิปหลักฐาน ระหว่างเรือกำลังเติมน้ำมัน ซึ่ง แตงโม และ กระติก กำลังหวีผมบนเรือ พร้อมระบุว่า การหวีผมต้องมีรากผม หรือ ไม่มีรากผมบนเรือ จากนั้น ได้เปิดไทม์ไลน์การเก็บหลักฐานของเจ้าหน้าที่พบว่า การเก็บหลักฐานครั้งแรกวันที่ 26 ก.พ. ไม่เจอเส้นผม ครั้งที่ 2 เก็บหลักฐานวันที่ 27 ก.พ. ไม่พบเส้นผม และหารอยเลือดจากนั้น วันที่ 1 มี.ค. พนักงานพิสูจน์หลักฐานเข้ามาเก็บหลักฐานตามที่พนักงานสอบสวนสั่ง พบเส้นผมจำนวนหนึ่ง โดยพบเส้นผม 1 เส้นที่ท้ายเรือบริเวณที่กาบเรือส่วนใต้น้ำ เป็นที่น่าสังเกตว่า ทำไมเส้นผมทุกเส้นที่พบอยู่ท้ายเรือ และทำไมจึงไม่พบในการเก็บเส้นผมครั้งแรกทั้งที่เจ้าหน้าที่จำนวนมาก คิดว่าเป็นไปได้หรือไม่“ผมมีหลักฐานยืนยันว่า แตงโม ตกหัวเรือ มีหลักฐานนิติวิทยาศาสตร์อีก 2 ชิ้น ว่า แซนนั่งอยู่ตำแหน่งไหนของเรือกันแน่ และมีกล้องวงจรปิดที่แซนนั่งอยู่ท้ายเรือ 22.32 น. 14 วินาที แต่จริงๆ นั่งอยู่ท้ายเรือ 22.06 น.ตามที่ปรากฏภาพในโหนกระแส ซึ่งเหตุการณ์ครั้งนี้ ตำรวจเอาแซนมาเป็นตัวตั้งในการทำคดี โดยพบว่า มีภาพวงจรปิดเวลาเดียวกับตำรวจ 22.32.10 น. แตงโม มานั่งท้ายเรือยองๆ แต่ไม่ได้ตกเรือ โดยเดินกลับเข้าไปในเรือ โดยพี่ชายแตงโม ดายศ ได้เห็นและเข้าใจแล้ว” นายอัจฉริยะ กล่าวทั้งนี้ นายอัจฉริยะ ได้เปิดคลิป ซึ่งเคยมีการเปิดเผยไปแล้วก่อนหน้านี้ เป็นบริเวณ กฝผ. โดยอธิบายว่า บริเวณท้ายเรือ มีเงาขยับตามท้ายเรือไปเรื่อย จากนั้นเงาได้หายไปจากบริเวณท้ายเรือ ซึ่งมีความแตกต่างกัน เมื่อมีอะไรตกน้ำตามคำให้การ ต้องตกลงไปด้านซ้าย แต่เงามีลักษณะหายเข้าไปในเรือ ทำให้พบได้ว่า ไม่เป็นไปตามที่แซนพูด ว่า แตงโมตกเรือ และยังพบว่ามีไฟในเรือหนึ่งดวง หากมีคนเดินผ่านไฟจะหายไป ซึ่งก็พบว่าไฟดวงนั้นหายไปสอดคล้องกับจังหวะที่แตงโมเดินเข้าไปบนเรือ“พรุ่งนี้จะเดินทางไปแจ้งดำเนินคดีกับเจ้าหน้าที่สอบสวนกองบัญชาการตำรวจภูธรภาค 1 ทั้งหมด ที่ปล่อยให้มีการสร้างหลักฐานอันเป็นเท็จสอดคล้องกับคำให้การคนบนเรือ และในวันที่ 26 เม.ย.จะมีการแถลงและเปิดหลักฐานในวันเดียวกับที่มีการแถลงปิดคดี เพื่อให้รู้ว่า การที่แตงโม ตกหัวเรือใครเป็นคนทำ มีแพทย์ชื่อดังของไทยยืนยันได้ว่าบาดแผลนี้ตกที่หัวเรือ และมีหลักฐานดีเอ็นเอ ว่า แตงโม อยู่หัวเรือตลอด ส่วนแซนอยู่ตรงไหนของเรือ ซึ่งพบว่าแซนมานั่งหลังคนขับเรือเพราะเจ็บตา แสดงว่า แซน โกหกมาตลอด สิ่งที่ทำเพื่อความยุติธรรมของแตงโม ไม่ใช่ว่า จะต้องเป็นดาราที่เสียชีวิตเพราะไปปัสสาวะท้ายเรือ” นายอัจฉริยะ กล่าว นายอัจฉริยะ กล่าวว่า มั่นใจว่าคดีดังกล่าวเป็นการฆาตกรรมที่ เกิดจากความเมาและมีอารมณ์ทางเพศ ไม่ใช่อุบัติเหตุ ซึ่งสอดคล้องกับกล้องวงจรปิดในเวลา 22.30 น. มีผู้ชายคนหนึ่งที่ตัวเองนั้นไม่ได้มีฐานะรวย แต่ภรรยารวย ได้เดินไปซ้อนทางข้างหลังของแตงโม ซึ่งช่วงเวลานั้นเป็นช่วงเวลาที่แตงโมตกเรือ"

X_test_arr = []
X_test_arr.append(test)
X_test = np.array(X_test_arr)

In [68]:
X_test

array(['อัจฉริยะ งัดหลักฐานฟันธงคดีแตงโม ฆาตกรรม ไม่ได้ตกจากท้ายเรือ เมื่อเวลา 14.00 น. ที่ รร.อัณณา นาวา ปากเกร็ด นายอัจฉริยะ เรืองรัตนพงษ์ จัดแถลงข่าว กรณีเปิดหลักฐาน แตงโม ไม่ได้ตกท้ายเรือ และข้อพิสูจน์ว่าตำรวจ มีความผิด ฐานปฏิบัติหน้าที่ไม่ชอบ ม.157 ว่า ขณะนี้ตำรวจยังไม่สามารถตอบเรื่องบาดแผลข้างขวา ซึ่งเคยออกมาระบุว่า ไม่ใช่สาเหตุการเสียชีวิตได้ โดย พ.อ.นพ. ธวัชชัย กาญจนรินทร์” อดีตศัลยแพทย์โรงพยาบาลพระมงกุฎเกล้า ได้วิเคราะห์ สรุปได้ว่า เกิดจากของที่คมมาก หรือ ถ้าเกิดในแม่น้ำจะพุ่งกระจายหยุดยาก ชมรมฯ เห็นด้วยกับบทความนี้ ว่า แผลขวาด้านใน ไม่สามารถเกิดจากใบพัดหรือฟินเรือได้ทั้งนี้ นายอัจฉริยะ ได้เปิดคลิปหลักฐาน ระหว่างเรือกำลังเติมน้ำมัน ซึ่ง แตงโม และ กระติก กำลังหวีผมบนเรือ พร้อมระบุว่า การหวีผมต้องมีรากผม หรือ ไม่มีรากผมบนเรือ จากนั้น ได้เปิดไทม์ไลน์การเก็บหลักฐานของเจ้าหน้าที่พบว่า การเก็บหลักฐานครั้งแรกวันที่ 26 ก.พ. ไม่เจอเส้นผม ครั้งที่ 2 เก็บหลักฐานวันที่ 27 ก.พ. ไม่พบเส้นผม และหารอยเลือดจากนั้น วันที่ 1 มี.ค. พนักงานพิสูจน์หลักฐานเข้ามาเก็บหลักฐานตามที่พนักงานสอบสวนสั่ง พบเ

# Initialize the tokenizer

In [69]:
# !pip install sentencepiece
from transformers import XLMRobertaTokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained(XLMR_chosen_premodel)

def preprocessing_for_bert(data):

    input_ids = []
    attention_masks = []

    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            #text=text_preprocessing(sent),  
            text=sent,
            add_special_tokens=True,        
            max_length=MAX_LEN,
            truncation=True,             
            padding='max_length',         
            #return_tensors='pt',           
            return_attention_mask=True      
        )
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

# Convert dataset into dataloader for inserting into the model

In [70]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

X_test_new = np.array([text for sub in X_test for text in sub]) 
test_inputs, test_masks = preprocessing_for_bert(X_test_new)
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

# Load the classification model

In [71]:
import torch.nn.functional as F
from transformers import XLMRobertaForSequenceClassification
import torch.nn as nn
import torch
bert_classifier_load = XLMRobertaForSequenceClassification.from_pretrained(XLMR_chosen_premodel,num_labels=8,output_attentions=False,output_hidden_states=False)
bert_classifier_load = nn.DataParallel(bert_classifier_load)
bert_classifier_load.load_state_dict(torch.load(XLMR_path))
bert_classifier_load.to(device)
bert_classifier_load.eval()

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            outputs = model(b_input_ids, b_attn_mask)
            logits = outputs.logits
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = all_logits.sigmoid().cpu().numpy()

    return probs

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out

# Predict News

In [72]:
probs = bert_predict(bert_classifier_load, test_dataloader)

In [73]:
probs

array([[5.2567478e-04, 5.2935746e-03, 1.3111026e-03, ..., 4.5257844e-03,
        8.1326440e-03, 8.7863529e-01],
       [2.5050247e-03, 5.5101127e-03, 6.5621696e-03, ..., 1.1891999e-02,
        5.6778654e-03, 6.5380538e-01],
       [5.4998900e-04, 6.1949976e-03, 7.5371540e-03, ..., 1.0085372e-02,
        1.1837160e-03, 7.0207477e-01],
       ...,
       [5.2194390e-04, 3.5453576e-03, 5.6004792e-04, ..., 2.0251847e-03,
        1.5820047e-03, 9.7961253e-01],
       [4.5726233e-04, 1.5954101e-03, 7.3461840e-04, ..., 2.0562010e-03,
        2.4410982e-03, 8.7101740e-01],
       [5.2099797e-04, 5.2544707e-03, 1.3010931e-03, ..., 4.4945250e-03,
        8.0766706e-03, 8.7923527e-01]], dtype=float32)

# Modify the threshold [Non-Crime Part]

In [74]:
threshold_set = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.1]

probs[...,0] = np.where(probs[...,0] > threshold_set[0],1,0)
probs[...,1] = np.where(probs[...,1] > threshold_set[1],1,0)
probs[...,2] = np.where(probs[...,2] > threshold_set[2],1,0)
probs[...,3] = np.where(probs[...,3] > threshold_set[3],1,0)
probs[...,4] = np.where(probs[...,4] > threshold_set[4],1,0)
probs[...,5] = np.where(probs[...,5] > threshold_set[5],1,0)
probs[...,6] = np.where(probs[...,6] > threshold_set[6],1,0)
probs[...,7] = np.where(probs[...,7] > threshold_set[7],1,0)

In [75]:
probs

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

# Create Dataframe for creating csv file

In [76]:
df_predict = pd.DataFrame(probs,columns=['pred_gambling','pred_murder','pred_sexual_abuse','pred_theft/burglary','pred_drug','pred_battery/assault','pred_accident','pred_non-crime'])
df_combined = pd.concat([df_crime_test,df_predict],axis=1)

In [77]:
df_combined.to_csv('./Result.csv',index=False)